Experiment Objective

Terms of Service

The TOS didn't limit the use of data. It mostly stated that the licensing was public domain, so the data could be used and linked to freely. However, the TOS reqested that the data be cited. Here is the APA citation they provided:

Emergency Medical Services Department. (2014). EMS - Incidents by Month [Data set]. City of Austin, Texas Open Data Portal. https://doi.org/10.26000/001.000001

The TOS also stated that they reserved the right to modify or discontinue the data provided at any time. It also stated that linked websites were provided as a convience, and they couldn't guarentee the provided websites' content. 

I think the most important request in the TOS was that personal information shouldn't be extracted from the dataset. Measures had been taken to remove personal data, but it could be possible to identify people by some of the features. Basically, don't use the data to dox people. 


Ethical Considerations

As this dataset is about animals, it doesn't put into question the demographic considerations envolved when the dataset envolves people. 

Model

This model is attempting to predict the subtype among the subset of animals that were euthenised. 

This dataset was selected becausse it is about the largest no-kill shelter is the US. It has a robust number of records. 

Data Collection

In [2]:
!pip3 install sodapy

In [13]:
import pandas as pd
import numpy as np
from sodapy import Socrata

client = Socrata("data.austintexas.gov", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9t4d-g238", limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [4]:
df.sample(2)


,animal_id,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name,outcome_subtype
317,A821155,2020-08-27T09:36:00.000,2020-08-27T09:36:00.000,2020-06-26T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair,Brown Tabby,*Toka,Foster
72,A821745,2020-09-05T14:59:00.000,2020-09-05T14:59:00.000,2019-08-18T00:00:00.000,Transfer,Dog,Neutered Male,1 year,Shiba Inu Mix,Tan/White,*Teddy,Partner


Data Preprocessing

In [15]:
#Start by checking for NaN
df = df[df.outcome_type == 'Euthanasia']
pd.isnull(df).sum()

animal_id             0
datetime              0
monthyear             0
date_of_birth         0
outcome_type          0
animal_type           0
sex_upon_outcome      0
age_upon_outcome      0
breed                 0
color                 0
name                146
outcome_subtype       2
dtype: int64

In [16]:
df[pd.isnull(df.outcome_subtype)]

,animal_id,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name,outcome_subtype
611,A821648,2020-08-16T11:38:00.000,2020-08-16T11:38:00.000,2019-08-16T00:00:00.000,Euthanasia,Other,Unknown,1 year,Raccoon,Gray,NaN,NaN
1518,A820140,2020-07-13T16:40:00.000,2020-07-13T16:40:00.000,2020-04-13T00:00:00.000,Euthanasia,Other,Unknown,2 months,Raccoon,Brown/Black,NaN,NaN


In [17]:
#since there is only 2 NaNs I decided to drop them
df = df.dropna(subset=['outcome_subtype'])

In [18]:
#One hot encoding for categorical variables
sex_coded = pd.get_dummies(df['sex_upon_outcome'], drop_first = True)

df = df.drop('sex_upon_outcome',axis = 1)

df = df.join(sex_coded)

In [19]:
animal_coded = pd.get_dummies(df['animal_type'], drop_first = True)

df = df.drop('animal_type',axis = 1)

df = df.join(animal_coded)

In [21]:
breed_coded = pd.get_dummies(df['breed'], drop_first = True)

df = df.drop('breed',axis = 1)

KeyError: 'breed'

In [23]:
breed_coded.sample(2)

,Bat,Black/Tan Hound,Boxer,Bull Terrier Mix,Chicken,Chihuahua Shorthair,Coyote,Domestic Longhair,Domestic Medium Hair,Domestic Shorthair,Domestic Shorthair Mix,Fox,Opossum,Pit Bull,Pit Bull Mix,Raccoon,Skunk,Squirrel,Whippet,Wolf Hybrid Mix
1805,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
622,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [24]:
df = df.join(breed_coded)

In [25]:
color_coded = pd.get_dummies(df['color'], drop_first = True)

df = df.drop('color',axis = 1)

df = df.join(color_coded)

In [26]:
#There is only one record for outcome_subtype == to Medical so I am dropping it 
df = df[df['outcome_subtype'] != 'Medical']


In [27]:
outcomes_mapping = {label: idx for idx, label in enumerate(np.unique(df['outcome_subtype']))}
outcomes_mapping

{'At Vet': 0,
 'Behavior': 1,
 'Court/Investigation': 2,
 'Rabies Risk': 3,
 'Suffering': 4,
 'Underage': 5}

In [28]:
df['outcome_subtype'] = df['outcome_subtype'].map(outcomes_mapping)
df.sample(2)

,animal_id,datetime,monthyear,date_of_birth,outcome_type,age_upon_outcome,name,outcome_subtype,Intact Male,Neutered Male,...,Red/Brown,Red/Gray,Tan,Tan/White,Torbie,Tortie/Black,Tricolor,White/Black,White/Brown Tabby,White/Gray
540,A821624,2020-08-19T13:05:00.000,2020-08-19T13:05:00.000,2018-08-15T00:00:00.000,Euthanasia,2 years,Fox,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1826,A819426,2020-06-29T09:40:00.000,2020-06-29T09:40:00.000,2019-06-29T00:00:00.000,Euthanasia,1 year,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
def conversion(time):
    yearList = time.split()
    if 'year' in time:
        return(int(yearList[0])*365)
    elif 'month' in time:
        return(int(yearList[0])*30)
    elif 'week' in time:
        return(int(yearList[0])*7)
    elif 'day' in time:
        return(yearList[0])
    else:
        return(np.nan)

df['age_upon_outcome_converted'] = df['age_upon_outcome'].apply(conversion)

In [30]:
df.sample(2)


,animal_id,datetime,monthyear,date_of_birth,outcome_type,age_upon_outcome,name,outcome_subtype,Intact Male,Neutered Male,...,Red/Gray,Tan,Tan/White,Torbie,Tortie/Black,Tricolor,White/Black,White/Brown Tabby,White/Gray,age_upon_outcome_converted
789,A821352,2020-08-10T10:51:00.000,2020-08-10T10:51:00.000,2019-08-10T00:00:00.000,Euthanasia,1 year,NaN,4,0,0,...,0,0,0,0,0,0,0,0,0,365
1254,A820533,2020-07-22T17:30:00.000,2020-07-22T17:30:00.000,2012-07-22T00:00:00.000,Euthanasia,8 years,NaN,3,1,0,...,0,0,0,0,0,0,1,0,0,2920


In [31]:
#I decided to drop the rows without values for the ages as well as there were only 10. This didn't get dropped 
#before because it was coded as 'NULL' instead of NaN

df = df.dropna(subset=['age_upon_outcome_converted'])

#I also decided to not use the name column becuase there are so many values and missing cells

df = df.drop('name', axis = 1)
df = df.drop('outcome_type', axis =1)
df = df.drop('age_upon_outcome', axis =1)
df = df.drop('animal_id', axis =1)
df = df.drop(['datetime','monthyear','date_of_birth'], axis =1)             

In [32]:
df = df[[col for col in df if col not in ['outcome_subtype']] 
       + ['outcome_subtype']]

In [41]:
print(len(df[df.outcome_subtype == 1]))
print(len(df[df.outcome_subtype == 2]))
print(len(df[df.outcome_subtype == 3]))
print(len(df[df.outcome_subtype == 4]))
print(len(df[df.outcome_subtype == 5]))

1
3
87
50
7


In [42]:
#We need to drop the row that is y ==1 because it can't be represented in both the train and test

df = df[df.outcome_subtype != 1]

In [43]:
#Scaling
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X, y = df.iloc[:, 0:-1].values, df.iloc[:, -1].values.astype(float)

X = stdsc.fit_transform(X).astype(float)


In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)

In [45]:
#Feature Selection
from sklearn.ensemble import RandomForestClassifier

feat_labels = df.columns[0:-1]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

X_train_subset = pd.DataFrame()
X_test_subset = pd.DataFrame()


for f in range(7):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))
    X_train_subset[str(f)] = X_train[:,indices[f]]
    X_test_subset[str(f)] = X_test[:,indices[f]]
    
X_test_subset_7 = np.array(X_test_subset)
X_train_subset_7 = np.array(X_train_subset)


 1) age_upon_outcome_converted     0.205780
 2) Bat                            0.087270
 3) Other                          0.078912
 4) Cat                            0.052933
 5) Domestic Shorthair             0.046850
 6) Unknown                        0.045000
 7) Gray/Gray                      0.039766


In [46]:
#Knearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
#First for all the data
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.75

In [47]:
#Now for just the seven selected features
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_subset_7, y_train)

knn.score(X_test_subset_7, y_test)



0.6666666666666666

In [48]:
#Now for 8 features

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

X_train_subset = pd.DataFrame()
X_test_subset = pd.DataFrame()


for f in range(8):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))
    X_train_subset[str(f)] = X_train[:,indices[f]]
    X_test_subset[str(f)] = X_test[:,indices[f]]
    
X_test_subset = np.array(X_test_subset)
X_train_subset = np.array(X_train_subset)

 1) age_upon_outcome_converted     0.205780
 2) Bat                            0.087270
 3) Other                          0.078912
 4) Cat                            0.052933
 5) Domestic Shorthair             0.046850
 6) Unknown                        0.045000
 7) Gray/Gray                      0.039766
 8) Opossum                        0.039493


In [49]:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_subset, y_train)

knn.score(X_test_subset, y_test)


0.6666666666666666

In [50]:
#Now for 6 features
forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

X_train_subset = pd.DataFrame()
X_test_subset = pd.DataFrame()


for f in range(6):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))
    X_train_subset[str(f)] = X_train[:,indices[f]]
    X_test_subset[str(f)] = X_test[:,indices[f]]
    
X_test_subset = np.array(X_test_subset)
X_train_subset = np.array(X_train_subset)

 1) age_upon_outcome_converted     0.205780
 2) Bat                            0.087270
 3) Other                          0.078912
 4) Cat                            0.052933
 5) Domestic Shorthair             0.046850
 6) Unknown                        0.045000


In [51]:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_subset, y_train)

knn.score(X_test_subset, y_test)

0.6458333333333334

Incresing the number of features to 8 didn't improve the score, and decreasing to 6 did lower the score as well. Therefore, 7 seems to be the correct number of features. <- This was for the original data set I used, but it has been updated since then and i don't think is correct anymore. 

In [27]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter = 1000)
lr.fit(X_train_subset_7, y_train)
lr.score(X_test_subset_7, y_test)


0.7090909090909091

This model didn't seem to perform very well. I don't think that the data is linearily seperable, so I initially recieved an error and had to increaes the number of iterations to get it to work. 

In [28]:
#Support Vector 
from sklearn.svm import SVC

svm = SVC(kernel='rbf', random_state=1, gamma=0.10, C=10.0)
svm.fit(X_train_subset_7, y_train)
svm.score(X_test_subset_7, y_test)

0.7454545454545455

This model performed slightly better, but it still isn't very accurate. Overall, it was better than the other two models. 

Experiment Summary

Data was collected using the API and instructions found at this webpage: https://dev.socrata.com/foundry/data.austintexas.gov/gjtj-jt2d

It was very easy to use. 

Preprocessing the data envolved a lot of one hot encoding to transform the features. There was also some missing data that had to be handled. 

I compared the K nearest neighbors, Logistic Regression, and kernal svm methods. I don't think any of them worked that well because some of the sub-types only had 3 or so examples in the data. I think if I had narrowed down my dependent variable to just the rabies-risk and aggression categories, it would have worked a lot better. In summary, I don't think I was able to create an effective model, but I think it's a very interesting dataset and there is potential for a good model. I learned more about data requirements for creating models. 